# EEP/IAS 118 - Section 6

## Fixed Effects Regression

### July 29, 2021

Today we will practice with fixed effects regressions in __R__. We have two different ways to estimate the model, and we will see how to do both and the situations in which we might favor one versus the other.

Let's give this a try using the dataset `wateruse.dta`, a panel of residential water use for residents in Alameda and Contra Costa Counties. The subset of households are high water users, people who used over 1,000 gallons per billing cycle. We have information on their water use, weather during the period, as well as information on the city and zipcode of where the home is located, and information on the size and value of the house.

Suppose we are interested in running the following panel regression of residential water use:

$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} ~~~~~~~~~~~~~~~~~~~~~~~(1)$$

Where $GPD$ is the gallons used per day by household $i$ in billing cycle $t$, $degree\_days$ the count of degree days experienced by the household in that billing cycle (degree days are a measure of cumulative time spent above a certain temperature threshold), and $precip$ the amount of precipitation in millimeters.

In [ ]:
library(tidyverse)
library(haven)
library(lfe)
waterdata <- read_dta("wateruse.dta") %>%
    mutate(gpd = (unit*748)/num_days)
waterdata <- mutate(waterdata,
          n = 1:nrow(waterdata))
head(waterdata)
dim(waterdata)
reg1 <- lm(gpd ~ degree_days + precip, data = waterdata)
summary(reg1)

Here we obtain an estimate of $\hat\beta_1 = 0.777$, telling us that an additional degree day per billing cycle is associated with an additional $0.7769$ gallon used per day. These billing cycles are roughly two months long, so this suggests an increase of roughly 47 gallons per billing cycle. Our estimate is statistically significant at all conventional levels, suggesting residential water use does respond to increased exposure to high heat.

We estimate a statistically insignificant coefficient on additional precipitation, which tells us that on average household water use in our sample doesn't adjust to how much it rains.

We might think that characteristics of the home impact how much water is used there, so we add in some home controls:


$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it}  + \beta_3 lotsize_{i} + \beta_4 homesize_i + \beta_5 num\_baths_i + \beta_6 num\_beds_i + \beta_7 homeval_i~~~~~~~~~~~~~~~~~~~~~~~(2)$$

In [ ]:
reg2 <- lm(gpd ~ degree_days + precip + lotsize + homesize + num_baths + num_beds + homeval, data = waterdata)
summary(reg2)

Our coefficient on $degree\_days$ remains statistically significant and doesn't change much, so we find that $\hat\beta_1$ is robust to the addition of home characteristics. Of these characteristics, we obtain statistically significant coefficients on the size of the lot (in acres), the size of the home ($ft^2$), and the number of bedrooms in the home.

We get a curious result for $\hat\beta_6$: for each additional bedroom in the home we predict that water use will _fall_ by 48 gallons per day. 

### Discussion: what might be driving this effect? 

In [ ]:
waterdata %>%
    filter( city <= 9) %>%
ggplot(aes(x=num_beds, y=gpd)) +
geom_point() +
    facet_grid(. ~ city)

waterdata %>%
    filter(city> 9 & city <= 18) %>%
ggplot(aes(x=num_beds, y=gpd)) +
geom_point() +
    facet_grid(. ~ city)

waterdata %>%
    filter( city> 18) %>%
ggplot(aes(x=num_beds, y=gpd)) +
geom_point() +
    facet_grid(. ~ city)

Since there are likely a number of sources of omitted variable bias in the previous model, we think it might be worth including some fixed effects in our model. 

## Method 1: Fixed Effects with lm() 

Up to this point we have been running our regressions using the `lm()` function. We can still use `lm()` for our fixed effects models, but it takes some more work.

Recall that we can write our general panel fixed effects model as 

$$ y_{it} = \beta x_{it} + \mathbf{a}_i + {d}_t + u_{it} $$

* $y$ our outcome of interest, which varies in both the time and cross-sectional dimensions
* $x_{it}$ our set of time-varying unit characteristics
* $\mathbf{a}_i$ our set of unit fixed effects
* $d_t$ our time fixed effects

We can estimate this model in `lm()` provided we have variables in our dataframe that correspond to $a_i$ and $d_t$. This means we'll have to generate them before we can run any regression.

### Generating Dummy Variables

In order to include fixed effects for our regression, we have to first generate the set of dummy variables that we want. For example, if we want to include a set of city fixed effects in our model, we need to generate them.

We can do this in a few ways.

1. First, we can use `mutate()` and add a separate line for each individual city:

In [ ]:
fe_1 <- waterdata %>%
    mutate(city_1 = as.numeric((city==1)),
           city_2 = as.numeric((city ==2)),
           city_3 = as.numeric((city ==3))) %>%
    select(n, city, city_1, city_2, city_3)
head(fe_1)

This can be super tedious though when we have a bunch of different levels of our variable that we want to make fixed effects for. In this case, we have 27 different cities.

2. Alternatively, we can use the `spread()` function to help us out. Here we add in a constant variable `v` that is equal to one in all rows, and a copy of city that adds "city_" to the front of the city number. Then we pass the data to `spread`, telling it to split the variable `cty` into dummy variables for all its levels, with all the "false" cases filled with zeros.



In [ ]:
fe_2 <- waterdata %>%
select(n, city)

head(fe_2)

fe_2 %>%
   mutate(v = 1, cty = paste0("city_", city)) %>% 
    spread(cty, v, fill = 0)

That is much easier! Let's now do that so that they'll be in `waterdata`:

In [ ]:
waterdata <- waterdata %>%
   mutate(v = 1, cty = paste0("city_", city)) %>% 
    spread(cty, v, fill = 0)
head(waterdata)
names(waterdata)

Note that both of the variables we used in `spread` are no longer in our dataset.

While we're at it, let's also add in a set of billing cycle fixed effects.

In [ ]:
waterdata <- waterdata %>%
   mutate(v = 1, cyc = paste0("cycle_", billingcycle)) %>% 
    spread(cyc, v, fill = 0)
head(waterdata)


Now we have all our variables to run the regression


$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it}  + \mathbf{a}_i + \mathbf{d}_t~~~~~~~~~~~~~~~~~~~~~~~(2)$$

Where $\mathbf{a}_i$ are our city fixed effects, and $\mathbf{d}_t$ our billing cycle fixed effects.

Now we can run our model! Well, now we can _write out_ our model. The challenge here is that we need to specify all of the dummy variables in our formula. We could do this all by hand, but when we end up with a bunch of fixed effects it's easier to use the following trick: we can write `y ~ .` to tell __R__ we want it to put every variable in our dataset other than $y$ on the right hand side of our regression. That means we can create a version of our dataset with only $gpd$, $degree\_days$, $precip$, and our fixed effects and won't have to write out all those fixed effects by hand!

Note that we can use `select()` and `-` to remove variables from our dataframe. If there is a list of variables in order that we want to get rid of, we can do it easily by passing a vector through! For instance, we want to get rid of the first 12 variables in our data, so we can add `-unit:-hh` in select to get rid of them all at once. if we separate with a comma, we can drop other sections of our data too!

In [ ]:
fe_data <- waterdata %>%
    select(-unit:-hh, -city, -n) 
head(fe_data)

fe_reg1 <- lm(gpd ~ ., data = fe_data)
summary(fe_reg1)

Since I specified it this way, __R__ chose the last dummy variable for each set of fixed effect to leave out as our omitted group. 

Now that we account for which billing cycle we're in (i.e. whether we're in the winter or whether we're in the summer), we find that the coefficient on $degree\_days$ is now much smaller and statistically insignificant. This makes sense, as we were falsely attributing the extra water use that comes from seasonality to temperature on its own. Now that we control for the season we're in via billing cycle fixed effects, we find that deviations in temperature exposure during a billing cycle doesn't result in dramatically higher water use within the sample.

### Discussion: Why did we drop the home characteristics from our model?

## Method 2: Fixed Effects with felm()

Alternatively, we could do everything way faster using the `felm()` function from the package __lfe__. This package doesn't require us to produce all the dummy variables by hand. Further, it performs the background math way faster so will be much quicker to estimate models using large datasets and many variables.

The syntax we use is now 

`felm(y ~ x1 + x2 + ... + xk | FE_1 + FE_2, data = df)`

* The first section $y \sim x1 + x2 +... xk$ is our formula, written the same way as with `lm()`
* We now add a `|` and in the second section we specify our fixed effects. Here we say $FE\_1 + FE\_2$ which tells __R__ to include fixed effects for each level of $FE\_1$ and $FE\_2$.
* Note that our fixed effect variables must be of class "factor" - we can force our variables to take this class by adding them as `as.factor(FE_1) + as.factor(FE_2)`.
* we add the data source after the comma, as before.

Let's go ahead and try this now with our water data model:

In [ ]:
fe_reg2 <- felm(gpd ~ degree_days + precip | as.factor(city) + as.factor(billingcycle), data = waterdata)
summary(fe_reg2)

And we estimate the exact same coefficients on $degree\_days$ and $precip$ as in the case where we specified everything by hand! We didn't have to mutate our data or add any variables. The one potential downside is that this approach doesn't report the fixed effects themselves. However, the tradeoff is that `felm` runs a lot faster than `lm`. To see this, we can compare run times:

In [ ]:
lm_start <- Sys.time()
fe_data <- waterdata %>%
   mutate(v = 1, cyc = paste0("cycle_", billingcycle)) %>% 
    spread(cyc, v, fill = 0) %>%
   mutate(v = 1, cty = paste0("city_", city)) %>% 
    spread(cty, v, fill = 0) %>%
    select(-unit:-hh, -city, -n) 
lm(gpd ~ ., data = fe_data)
lm_end <- Sys.time()
lm_dur <- lm_end - lm_start


felm_start <- Sys.time()
felm(gpd ~ degree_days + precip | as.factor(city) + as.factor(billingcycle), data = waterdata)
felm_end <- Sys.time()
felm_dur <- felm_end - felm_start

print(paste0("lm() duration is ", lm_dur, " seconds, while felm() duration is ", felm_dur, " seconds."))



Okay, neither of these models took very long, but that's because we only have two covariates other than our fixed effects and only around 2300 observations. If we have hundreds of covariates and millions of observations, this time difference becomes massive.


# Regression Discontinuity

Let's practice running a regression discontinuity model. Suppose we were interested in exploring the weird relationship we saw earlier between water use and number of bedrooms in a home. Let's take a look at that relationship a bit more closely.

In [ ]:

waterdata %>%
    ggplot(aes(x = num_beds, y = gpd)) +
    geom_point( alpha = 0.4, colour = "royalblue")

We see that average water use appears to rise as we add bedrooms to a house from a low number, peaks when households have five bedrooms, then begins to fall with larger and larger houses... though there are a few high outliers in the 6-9 bedroom cases as well, which might overshadow that trend.

Is there something else that's correlated with the number of bedrooms in a home that may also be driving this?

In [ ]:

waterdata %>%
    ggplot(aes(x = num_beds, y = lotsize)) +
    geom_point( alpha = 0.4, colour = "royalblue")

It looks like lotsize and the number of bedrooms share a similar relationship - lot size increasing in \# bedrooms up until 5, then declining from there.

Given that it looks like 5 bedrooms is where the relationship changes, let's use this as our running variable and allow the relationship for the number of bedrooms to differ around a threshold of five bedrooms. We can write an RD model as 

$$ GPD_{it} = \beta_0 + \beta_1 T_i + \beta_2 (num\_beds - 5) + \beta_3\left( T_i \times (num\_beds - 5) \right) + u_{it} $$

In [ ]:
rd_data <- waterdata %>%
    select(gpd, num_beds, lotsize) %>%
    mutate(treat = (num_beds > 5),
          beds_below = num_beds - 5,
          beds_above = treat * (num_beds - 5)) 

rd_reg <- lm(gpd ~ treat + beds_below + beds_above, data = rd_data)
summary(rd_reg)

What if we limit our comparison closer to around the threshold? Right now we're using data from the entire sample, but this might not be a valid comparison. Let's see what happens when we reduce our bandwidth to 3 and look at only homes with between 3 and 8 bedrooms.

In [ ]:
rd_data_trim <- rd_data %>%
    filter(!(num_beds < 2)) %>%
    filter(!(num_beds > 8))

rd_reg2 <- lm(gpd ~ treat + beds_below + beds_above, data = rd_data_trim)
summary(rd_reg2)

We now estimate a treatment effect at the discontinuity! Our model finds a discontinuity, estimating a jump down of 284 gallons per day right around the 5 bedroom threshold. 

However, we saw earlier that it appears that lotsize is correlated with the number of bedrooms in the home, and is definitely a factor correlated with residential water use. What happens to our LATE estimate when we control for lot size in our regression?

In [ ]:
rd_reg3 <- lm(gpd ~ treat + beds_below + beds_above + lotsize, data = rd_data_trim)
summary(rd_reg3)

Once we control for lot size, our interpretation changes. Now we estimate a coefficient on treatment nearly half the magnitude as before and without statistical significance. 

### Discussion Q: What does this tell us about the covariance between lot size and the number of bedrooms?

# Fixed Effects Practice Question #1

#### From a random sample of agricultural yields Y (1000 dollars per acre) for region $i$ in year $t$ for the US, we have estimated the following eqation:

 \begin{align*} \widehat{\log(Y)}_{it} &=	0.49	+ .01 GE_{it} ~~~~ 	R^2 = .32\\
	&~~~~~(.11)	 ~~~~ (.01)                ~~~~  n = 1526       \end{align*}

#### (a) Interpret the results on the Genetically engineered ($GE$) technology on yields. (follow SSS= Sign Size Significance)

#### (b) Suppose $GE$ is used more on the West Coast, where crop yields are also higher. How would the estimated effect of GE change if we include a West Coast region dummy variable in the equation? Justify your answer.

#### (c) If we include region fixed effects, would they control for the factors in (b)? Justify your answer.

#### (d)  If yields have been generally improving over time and GE adoption was only recently introduced in the USA, what would happen to the coefficient of GE if we included year fixed effects?



# Fixed Effects Practice Question #2

#### A recent paper investigates whether advertisement for Viagra causes increases in birth rates in the USA.  Apparently, advertising for products, including Viagra, happens on TV and reaches households that have a TV within a Marketing region and does not happen in areas outside a designated marketing region. What the authors do is look at hospital birth rates in regions inside and near the advertising region border  and collect data on dollars per 100 people (Ads) for a certain time, and compare those to the birth rates in hospitals located outside and near the advertising region designated border. They conduct a panel data analysis and estimate the following model:

$$ Births_{it} = \beta_0 + \beta_1 Ads + \beta_2 Ads^2 + Z_i + M_t + u_{it}$$

#### Where $Z_i$ are zipcode fixed effects and $M_t$ monthly fixed effects.

#### (a) Why do the authors include Zip Code Fixed Effects? In particular, what would be a variable that they are controlling for when adding Zip Code fixed effects that could cause a problem when interpreting the marginal effect of ad spending on birth rates? What would that (solved) problem be?

#### (b) Why do they add month fixed effects? 

